In [1]:
# The code was removed by Watson Studio for sharing.

# Capstone Project - The Battle of Neighborhoods - Week 4 Part 1

# Table of contents

Introduction: Business Problem

Data

Methodology

Analysis

Results and Discussion

Conclusion


# Introduction: Business Problem 
    
This project aims to select the safest borough in London based on the total crimes, explore the neighborhoods of that borough to find the 10 most common venues in each neighborhood and finally cluster the neighborhoods using k-mean clustering.
This report will be targeted to people who are looking to relocate to London. Inorder to finalise a neighborhood to hunt for an apartment, safety is considered as a top concern when moving to a new place. If you don’t feel safe in your own home, you’re not going to be able to enjoy living there. The crime statistics will provide an insight into this issue.
We will focus on the safest borough and explore its neighborhoods and the 10 most common venues in each neighborhood so that the best neighborhood suited to an individual's needs can be selected.

# Data 

Based on definition of our problem, factors that will influence our decision are:

-The total number of crimes commited in each of the borough during the last year.

-The most common venues in each of the neighborhood in the safest borough selected.

Following data sources will be needed to extract/generate the required information:


Part 1: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016: A dataset consisting of the crime statistics of each borough in London obtained from Kaggle

Part 2: Scraping additional information of the different Boroughs in London from a Wikipedia page.: More information regarding the boroughs of London is scraped using the Beautifulsoup library

Part 3: Creating a new dataset of the Neighborhoods of the safest borough in London and generating their co-ordinates.: Co-ordinate of neighborhood will be obtained using Google Maps API geocoding

# Part 1: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016


About this file

-lsoa_code: code for Lower Super Output Area in Greater London.

-borough: Common name for London borough.

-major_category: High level categorization of crime

-minor_category: Low level categorization of crime within major category.

-value: monthly reported count of categorical crime in given borough

-year: Year of reported counts, 2008-2016

-month: Month of reported counts, 1-12


Data set URL: https://www.kaggle.com/jboysen/london-crime

### Importing the required Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping  

#!conda install -c conda-forge geocoder --yes
import geocoder

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


In [3]:
CLIENT_ID = 'EG4F4RL1FDCK0XUJ2YRYDBZVKIQCYTATZD3KNBIVGXLJPKLA' # your Foursquare ID
CLIENT_SECRET = '5ULV3ADMR4HEIIK2OMKKGOJ0RKDWVI00CPHET0FS45B4MJYR' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EG4F4RL1FDCK0XUJ2YRYDBZVKIQCYTATZD3KNBIVGXLJPKLA
CLIENT_SECRET:5ULV3ADMR4HEIIK2OMKKGOJ0RKDWVI00CPHET0FS45B4MJYR


### Read in the dataset

In [4]:
# Read in the data 
# Fetch the file
my_file = project.get_file("london_crime_by_lsoa.csv")
my_file
# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
#import pandas as pd
df=pd.read_csv(my_file)
#df = pd.read_csv("london_crime_by_lsoa.csv")

In [5]:
# View the top rows of the dataset
df.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6


### Accessing the most recent crime rates (2016)

In [6]:
# Taking only the most recent year (2016) and dropping the rest
df.drop(df.index[df['year'] != 2016], inplace = True)

# Removing all the entires where crime values are null  
df = df[df.value != 0]

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

In [7]:
# Shape of the data frame
df.shape

(392042, 7)

In [8]:
# View the top of the dataset 
df.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01004177,Sutton,Theft and Handling,Theft/Taking of Pedal Cycle,1,2016,8
1,E01000733,Bromley,Criminal Damage,Criminal Damage To Motor Vehicle,1,2016,4
2,E01003989,Southwark,Theft and Handling,Theft From Shops,4,2016,8
3,E01002276,Havering,Burglary,Burglary in a Dwelling,1,2016,8
4,E01003674,Redbridge,Drugs,Possession Of Drugs,2,2016,11


### Change the column names¶

In [9]:
df.columns = ['LSOA_Code', 'Borough','Major_Category','Minor_Category','No_of_Crimes','Year','Month']
df.head()

,LSOA_Code,Borough,Major_Category,Minor_Category,No_of_Crimes,Year,Month
0,E01004177,Sutton,Theft and Handling,Theft/Taking of Pedal Cycle,1,2016,8
1,E01000733,Bromley,Criminal Damage,Criminal Damage To Motor Vehicle,1,2016,4
2,E01003989,Southwark,Theft and Handling,Theft From Shops,4,2016,8
3,E01002276,Havering,Burglary,Burglary in a Dwelling,1,2016,8
4,E01003674,Redbridge,Drugs,Possession Of Drugs,2,2016,11


In [10]:
# View the information of the dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392042 entries, 0 to 392041
Data columns (total 7 columns):
LSOA_Code         392042 non-null object
Borough           392042 non-null object
Major_Category    392042 non-null object
Minor_Category    392042 non-null object
No_of_Crimes      392042 non-null int64
Year              392042 non-null int64
Month             392042 non-null int64
dtypes: int64(3), object(4)
memory usage: 20.9+ MB
